In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cp
from rsome import ro
from rsome import grb_solver as grb
import rsome as rso
import numpy as np

import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

import pandas as pd
import sbibm

In [2]:
plt.rcParams['mathtext.fontset'] = 'stix'
plt.rcParams['font.family'] = 'STIXGeneral'

SMALL_SIZE = 12
MEDIUM_SIZE = 14
BIGGER_SIZE = 18

plt.rc('font', size=BIGGER_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=BIGGER_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=BIGGER_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=BIGGER_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [3]:
task_name = "gaussian_mixture"

task = sbibm.get_task(task_name)
prior = task.get_prior_dist()
simulator = task.get_simulator()

# used for setting up dimensions
sample_c = prior.sample((1,))
sample_x = simulator(sample_c)
n = sample_c.shape[-1] # number of items (c in R^n)
d = sample_x.shape[-1] # dim of context to utility (x in R^d)

In [4]:
theta = np.random.randint(low=0, high=2, size=(d, n)) # theta has to be globally fixed to ensure exchangeability

def g(x):
    c = (x @ theta) ** 2
    return c

In [18]:
N = 2_000
N_train = 1000
N_test  = 1000

c_dataset = prior.sample((N,)).numpy()
x_dataset = simulator(c_dataset).numpy()

device = ("cuda" if torch.cuda.is_available() else "cpu")
to_tensor = lambda r : torch.tensor(r).to(torch.float32).to(device)
x_train, x_cal = to_tensor(x_dataset[:N_train]), to_tensor(x_dataset[N_train:])
c_train, c_cal = to_tensor(c_dataset[:N_train]), to_tensor(c_dataset[N_train:])

c_test = prior.sample((N_test,)).numpy()
x_test = simulator(c_test).numpy()

In [19]:
train_dataset = TensorDataset(x_train, c_train)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

class FeedforwardNN(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dim=64):
        super(FeedforwardNN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [21]:
model = FeedforwardNN(input_dim=d, output_dim=n).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 1_000

for epoch in range(num_epochs):
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()  # Zero the gradients
        outputs = model(batch_X)  # Forward pass
        loss = criterion(outputs, batch_y)  # Compute the loss
        loss.backward()  # Backpropagation
        optimizer.step()  # Update weights

    if epoch % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [1/1000], Loss: 29.9002
Epoch [101/1000], Loss: 1.0652
Epoch [201/1000], Loss: 0.8301
Epoch [301/1000], Loss: 0.7555
Epoch [401/1000], Loss: 1.0764
Epoch [501/1000], Loss: 0.8217
Epoch [601/1000], Loss: 0.8016
Epoch [701/1000], Loss: 1.1656
Epoch [801/1000], Loss: 0.8754
Epoch [901/1000], Loss: 1.1227


In [22]:
def box_solve_generic(c_box_lb, c_box_ub, c_true, p, B):
    covered = int(np.all(c_box_lb <= c_true) and np.all(c_true <= c_box_ub))

    # perform RO over constraint region
    model = ro.Model()

    w = model.dvar(n)
    c = model.rvar(n)
    uset = (c_box_lb <= c, c <= c_box_ub)

    model.minmax(-c @ w, uset)
    model.st(w <= 1)
    model.st(w >= 0)
    model.st(p @ w <= B)

    model.solve(grb)
    return covered, model.get()

In [23]:
# *marginal* box constraint (i.e. just ignore contextual information)
def box_solve_marg(alpha, x, c_true, p, B):
    alpha = alpha / 20
    c_box_lb = np.quantile(c_dataset, q=(alpha / 2), axis=0)
    c_box_ub = np.quantile(c_dataset, q=(1 - alpha / 2), axis=0)
    return box_solve_generic(c_box_lb, c_box_ub, c_true, p, B)

In [24]:
# *conditional* box conformal constraint (PTC-B)
def box_solve_cp(alpha, x, c_true, p, B):
    c_pred = model(x_cal)
    box_cal_scores = np.linalg.norm((c_pred - c_cal).detach().cpu().numpy(), np.inf, axis=1)
    conformal_quantile = np.quantile(box_cal_scores, q=1 - alpha, axis=0)

    c_box_hat = model(to_tensor(x)).detach().cpu().numpy()
    c_box_lb = c_box_hat - conformal_quantile
    c_box_ub = c_box_hat + conformal_quantile
    return box_solve_generic(c_box_lb, c_box_ub, c_true, p, B)

In [25]:
def ellipsoid_solve_generic(c_ellipse_center, c_ellipse_axes, c_ellipse_cutoff, c_true, p, B):
    # perform RO over constraint region
    model = ro.Model()

    w = model.dvar(n)
    c = model.rvar(n)

    uset = rso.norm((c - c_ellipse_center).T @ c_ellipse_axes, 2) <= c_ellipse_cutoff
    covered = int(np.linalg.norm((c_true - c_ellipse_center).T @ c_ellipse_axes) <= c_ellipse_cutoff)

    model.minmax(-c @ w, uset)
    model.st(w <= 1)
    model.st(w >= 0)
    model.st(p @ w <= B)

    model.solve(grb)
    return covered, model.get()

In [26]:
# *marginal* ellipsoid constraint
def ellipsoid_solve_marg(alpha, x, c_true, p, B):
    mu = np.mean(c_dataset, axis=0)
    cov = np.cov(c_dataset.T)
    sqrt_cov_inv = np.linalg.cholesky(np.linalg.inv(cov))
    mah_dists = np.linalg.norm((c_dataset - mu) @ sqrt_cov_inv, axis=1)
    cutoff = np.quantile(mah_dists, q=1 - alpha)
    
    return ellipsoid_solve_generic(mu, sqrt_cov_inv, cutoff, c_true, p, B)

In [27]:
# *conditional* ellipsoid conformal constraint (PTC-E)
def ellipsoid_solve_cp(alpha, x, c_true, p, B):
    c_pred = model(x_cal)
    residuals = (c_pred - c_cal).detach().cpu().numpy()

    cov = np.cov(residuals.T)
    sqrt_cov_inv = np.linalg.cholesky(np.linalg.inv(cov))
    ellipsoid_cal_scores = np.linalg.norm(residuals @ sqrt_cov_inv, axis=1)
    conformal_quantile = np.quantile(ellipsoid_cal_scores, q=1 - alpha, axis=0)

    c_ellipsoid_hat = model(to_tensor(x)).detach().cpu().numpy()   
    return ellipsoid_solve_generic(c_ellipsoid_hat, sqrt_cov_inv, conformal_quantile, c_true, p, B)

In [28]:
alphas = [0.05]
name_to_method = {
    "Box": box_solve_marg,
    "PTC-B": box_solve_cp,
    "Ellipsoid": ellipsoid_solve_marg,
    "PTC-E": ellipsoid_solve_cp,
}
method_coverages = {r"$\alpha$": alphas}
method_values = {r"$\alpha$": alphas}

n_trials = 100

for method_name in name_to_method:
    for alpha in alphas:
        covered = 0
        values = []
        
        for trial_idx in range(n_trials):
            x = x_test[trial_idx]
            c_true = c_test[trial_idx]

            p = np.random.randint(low=0, high=1000, size=n)
            u = np.random.uniform(low=0, high=1)
            B = np.random.uniform(np.max(p), np.sum(p) - u * np.max(p))

            (covered_trial, value_trial) = name_to_method[method_name](alpha, x, c_true, p, B)
            covered += covered_trial
            values.append(value_trial)

        if method_name not in method_coverages:
            method_coverages[method_name] = []
            method_values[method_name] = []

        method_coverages[method_name].append(covered / n_trials)
        method_values[method_name].append(np.mean(values))

coverage_df = pd.DataFrame(method_coverages)
values_df = pd.DataFrame(method_values)

Restricted license - for non-production use only - expires 2024-10-28
Being solved by Gurobi...


Solution status: 2
Running time: 0.0010s
Being solved by Gurobi...
Solution status: 2
Running time: 0.0003s
Being solved by Gurobi...
Solution status: 2
Running time: 0.0003s
Being solved by Gurobi...
Solution status: 2
Running time: 0.0003s
Being solved by Gurobi...
Solution status: 2
Running time: 0.0003s
Being solved by Gurobi...
Solution status: 2
Running time: 0.0003s
Being solved by Gurobi...
Solution status: 2
Running time: 0.0003s
Being solved by Gurobi...
Solution status: 2
Running time: 0.0003s
Being solved by Gurobi...
Solution status: 2
Running time: 0.0003s
Being solved by Gurobi...
Solution status: 2
Running time: 0.0003s
Being solved by Gurobi...
Solution status: 2
Running time: 0.0003s
Being solved by Gurobi...
Solution status: 2
Running time: 0.0003s
Being solved by Gurobi...
Solution status: 2
Running time: 0.0003s
Being solved by Gurobi...
Solution status: 2
Running time: 0.0003s
Being solved by Gurobi...
Solution status: 2
Running time: 0.0003s
Being solved by Gurob

In [29]:
print(coverage_df.to_latex(index=False,
                  formatters={"name": str.upper},
                  float_format="{:.2f}".format,
                  column_format="c" * len(method_coverages),
))

print(values_df.to_latex(index=False,
                  formatters={"name": str.upper},
                  float_format="{:.2f}".format,
                  column_format="c" * len(method_values)
))

\begin{tabular}{ccccc}
\toprule
$\alpha$ & Box & PTC-B & Ellipsoid & PTC-E \\
\midrule
0.05 & 1.00 & 0.94 & 0.96 & 0.97 \\
\bottomrule
\end{tabular}

\begin{tabular}{ccccc}
\toprule
$\alpha$ & Box & PTC-B & Ellipsoid & PTC-E \\
\midrule
0.05 & 0.00 & -2.33 & 0.00 & -2.48 \\
\bottomrule
\end{tabular}

